import os
import shutil
from sklearn.model_selection import train_test_split

In [38]:
import os
import shutil
from sklearn.model_selection import train_test_split
import pandas as pd
from PIL import Image

In [43]:


def create_image_dataframe(base_dir):
    # List to hold the rows of the dataframe
    data = []
    
    # Loop over the 'train' and 'test' directories
    for split in ['train', 'test']:
        split_dir = os.path.join(base_dir, split)
        
        # Check if the split directory exists
        if not os.path.isdir(split_dir):
            print(f"Directory not found: {split_dir}")
            continue
        
        # Iterate over each impairment type directory within train/test
        for impairment_type in ['Moderate Impairment', 'Very Mild Impairment', 'Mild Impairment']:
            impairment_dir = os.path.join(split_dir, impairment_type)

            for img_name in os.listdir(impairment_dir):
                if img_name.lower().endswith('.jpg'):
                    # Full image path
                    img_path = os.path.join(impairment_dir, img_name)
                    
                    # Open the image and load it into memory as an object
                    with open(img_path, 'rb') as img_file:
                        img_data = img_file.read()  # Read the image file as bytes
                        img = Image.open(io.BytesIO(img_data))  # Open the image using Pillow
                    
                    # Append a new row with impairment type, image name, image path, and image data
                    data.append([impairment_type, img_name, img_path, img])  

    # Convert the list of data into a pandas DataFrame
    df = pd.DataFrame(data, columns=['Impairment_Type', 'Image_Name', 'Image_Path', 'Image_Data'])
    return df

# Define your base directory where 'train' and 'test' folders are located
base_dir = '/Users/selahmitchell/Downloads/alzheimer_detection'  # Ensure there is no trailing space

# Generate the dataframe
df = create_image_dataframe(base_dir)

# Display the first few rows
print(df.head())
df.info()


       Impairment_Type                                         Image_Name  \
0  Moderate Impairment  ModerateImpairment-802-_jpg.rf.4d240c4e9cd1071...   
1  Moderate Impairment  ModerateImpairment-926-_jpg.rf.d801d2fca121257...   
2  Moderate Impairment  ModerateImpairment-1289-_jpg.rf.4c1f14c4b0e529...   
3  Moderate Impairment  ModerateImpairment-1637-_jpg.rf.62fdb6b600da49...   
4  Moderate Impairment  ModerateImpairment-2204-_jpg.rf.4e44d25282395c...   

                                          Image_Path  \
0  /Users/selahmitchell/Downloads/alzheimer_detec...   
1  /Users/selahmitchell/Downloads/alzheimer_detec...   
2  /Users/selahmitchell/Downloads/alzheimer_detec...   
3  /Users/selahmitchell/Downloads/alzheimer_detec...   
4  /Users/selahmitchell/Downloads/alzheimer_detec...   

                                          Image_Data  
0  <PIL.JpegImagePlugin.JpegImageFile image mode=...  
1  <PIL.JpegImagePlugin.JpegImageFile image mode=...  
2  <PIL.JpegImagePlugin.JpegImageFi